In [ ]:
# Notebook pour la description de la composition des clusters

# Library ----------------------------------------------------------------
library(dplyr)
library(tidyr)
library(ggplot2)
library(sondr)
library(clessnize)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Charger les données et le résultat du clustering
kmeans_result <- readRDS("../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_kmeans.rds")
df_pilot1_2022 <- readRDS("../../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_2022.rds")

# Sélectionner les variables utilisées pour le clustering (exclure "id" et les colonnes commençant par "cluster")
variablesUsedForClustering <- names(df_pilot1_2022)[ !grepl("^cluster|^id$", names(df_pilot1_2022)) ]

# Créer une dataframe enrichie avec la variable cluster convertie en facteur
dfUsedForClustering <- df_pilot1_2022
dfUsedForClustering$cluster <- as.factor(kmeans_result$cluster)

# Définir le chemin de sauvegarde des graphiques
output_path <- "../../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/qc_2022"


In [ ]:
# Fonction pour calculer l'importance relative de chaque variable 
# dans la création des clusters -----------------------------

# Fonction pour calculer l'importance relative des variables dans le clustering
calculate_importance <- function(data, variables) {
  importance_scores <- sapply(variables, function(var) {
    overall_mean <- mean(data[[var]], na.rm = TRUE)
    cluster_means <- tapply(data[[var]], data$cluster, mean, na.rm = TRUE)
    abs_diff <- abs(cluster_means - overall_mean) / sd(data[[var]], na.rm = TRUE)
    return(mean(abs_diff, na.rm = TRUE))
  })
  importance_scores <- importance_scores / sum(importance_scores)  # Normaliser
  return(importance_scores)
}


In [ ]:
# Fonction pour décrire les personas en affichant les moyennes et écarts-types par cluster
# (La variable "cluster" doit déjà être présente et factorisée dans data)
describe_personas <- function(data, variables) {
  persona_summary <- data %>%
    group_by(cluster) %>%
    summarise(across(all_of(variables), list(mean = ~mean(.x, na.rm = TRUE),
                                               sd = ~sd(.x, na.rm = TRUE))))
  print(persona_summary)
}

In [ ]:
# Calcul du score d'importance relatif pour chaque variable
importance_scores <- calculate_importance(dfUsedForClustering, variablesUsedForClustering)
print(importance_scores)

# Afficher les statistiques descriptives par cluster
describe_personas(dfUsedForClustering, variablesUsedForClustering)

In [ ]:
# Initialiser une liste pour stocker tous les prompts
# Initialiser une liste pour stocker tous les prompts
prompts <- list()
all_data_viz <- data.frame()

for (cluster_value in levels(dfUsedForClustering$cluster)) {
  # Extraire les données du cluster
  cluster_data <- dfUsedForClustering %>% filter(cluster == cluster_value)
  
  # Calculer les moyennes par variable pour le cluster et globalement
  means <- sapply(variablesUsedForClustering, function(var) mean(cluster_data[[var]], na.rm = TRUE))
  overall_means <- sapply(variablesUsedForClustering, function(var) mean(dfUsedForClustering[[var]], na.rm = TRUE))
  
  # Identification des caractéristiques marquantes
  characteristic_vars <- sapply(variablesUsedForClustering, function(var) {
    if (all(dfUsedForClustering[[var]] %in% c(0, 1))) {
      prop_cluster <- mean(cluster_data[[var]], na.rm = TRUE)
      prop_overall <- mean(dfUsedForClustering[[var]], na.rm = TRUE)
      p_value <- prop.test(x = c(sum(cluster_data[[var]]), sum(dfUsedForClustering[[var]])),
                           n = c(nrow(cluster_data), nrow(dfUsedForClustering)))$p.value
      prop_diff <- abs(prop_cluster - prop_overall)
      if (p_value < 0.05 && prop_diff > 0.3) {
        return(ifelse(prop_cluster > prop_overall, "green", "red"))
      } else {
        return("black")
      }
    } else {
      t_test <- t.test(cluster_data[[var]], dfUsedForClustering[[var]])
      effect_size <- abs(t_test$estimate[1] - t_test$estimate[2]) / sd(dfUsedForClustering[[var]], na.rm = TRUE)
      if (t_test$p.value < 0.05 && effect_size > 0.25) {
        return(ifelse(t_test$estimate[1] > t_test$estimate[2], "green", "red"))
      } else {
        return("black")
      }
    }
  })
  
  # Calculer quelques statistiques démographiques pour le cluster
  perc_femmes     <- mean(cluster_data$female, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$langEn, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$langFr, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethn_White, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_hetero, na.rm = TRUE) * 100
  educ_mean       <- mean(cluster_data$educ, na.rm = TRUE)
  age_mean        <- mean(cluster_data$age, na.rm = TRUE)
  revenu_moyen    <- mean(cluster_data$ses_income, na.rm = TRUE)
  
  # Générer le prompt de suggestion de nom pour ce cluster
  prompt_description <- paste0(
    "Persona ", cluster_value, ":\n",
    "Sup: ", if (any(characteristic_vars == "green")) {
      paste(variablesUsedForClustering[characteristic_vars == "green"], collapse = ", ")
    } else {
      "Aucune"
    }, "\n",
    "Inf: ", if (any(characteristic_vars == "red")) {
      paste(variablesUsedForClustering[characteristic_vars == "red"], collapse = ", ")
    } else {
      "Aucune"
    }, "\n",
    "Femmes: ", round(perc_femmes, 1), "%, Immigrants: ", round(perc_immigrants, 1), "%\n",
    "Âge: ", round(age_mean, 1), " ans, Revenu: ", round(revenu_moyen, 1), " k$\n",
    "Langues: En ", round(perc_langEn, 1), "%, Fr ", round(perc_langFr, 1), "%, Autres: ", round(perc_ses_languageOther, 1), "%\n",
    "Édu: ", round(educ_mean, 1), " (0=bas, 1=élevé), Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
    "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n",
    "Proposez un nom (ex: Prénom: Descriptif) et une description courte."
  )
  
  # Stocker le prompt dans la liste
  prompts[[cluster_value]] <- prompt_description
  
  # Préparer les données pour le graphique (si besoin)
  data_viz <- data.frame(
    Variable = variablesUsedForClustering, 
    Mean = means, 
    Overall = overall_means, 
    Cluster = cluster_value,
    Color = characteristic_vars,
    Persona = paste("Persona", cluster_value)
  )
  all_data_viz <- rbind(all_data_viz, data_viz)
}


In [ ]:
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  print(paste("===== Prompt pour le cluster", cluster, "====="))
  print(prompts[[cluster]])
  
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  print(paste0("Réponse pour le cluster ", cluster, " :"))
  print(resp$content)
}

In [ ]:
# Faire le graphique pour voir les variables significatives par cluster

all_data_viz_signif <- all_data_viz %>%
  filter(Color %in% c("green", "red"))

ggplot(all_data_viz_signif, aes(x = reorder_within(Variable, Mean, Persona), 
                                y = Mean, fill = Color)) +
  geom_bar(stat = "identity", color = "black", show.legend = FALSE) +
  scale_fill_identity() +
  labs(title = "Visualisation des personas (Variables significatives)", x = "", y = "Moyenne / Proportion") +
  theme_minimal() +
  coord_flip() +
  facet_wrap(~ Persona, scales = "free_y") +
  scale_x_reordered()